In [1]:
import torch
import torch.nn as nn
import timm
import time
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import cv2
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, cohen_kappa_score, hamming_loss, confusion_matrix, precision_score, recall_score
import matplotlib.pyplot as plt
import json

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import seaborn as sns
from tqdm import tqdm


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Dispositivo: {device}")

# ==================== PATHS CORRETOS ====================

# Base do dataset
DATASET_BASE = "/kaggle/input/odir5k"
DATASET_ROOT = "/kaggle/input/odir5k/data/odir5k"

# Paths das splits
TRAIN_DIR = f"{DATASET_ROOT}/train"
VAL_DIR = f"{DATASET_ROOT}/val"
TEST_DIR = f"{DATASET_ROOT}/test"

# Output
MODEL_SAVE_PATH = "/kaggle/working"
CURRENT_DIR = "/kaggle/working"
      
class ODIRDataset(Dataset):
    """Dataset ODIR-5K usando estrutura de pastas"""
    
    def __init__(self, split='train', transform=None):
        self.split = split
        self.transform = transform
        self.img_dir = os.path.join(DATASET_ROOT, split)
        
        # Carregar metadados
        metadata_path = os.path.join(self.img_dir, f"{split}_metadata.csv")
        self.data = pd.read_csv(metadata_path)
        self.disease_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
        self.image_files = [f for f in os.listdir(self.img_dir) if f.endswith('.jpg')]
        
        print(f"  📂 {split}: {len(self.data)} pacients, {len(self.image_files)} images")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        patient_id = str(row['ID'])
        
        img_path = None
        for side in ['left', 'right']:
            img_name = f"{patient_id}_{side}.jpg"
            full_path = os.path.join(self.img_dir, img_name)
            if os.path.exists(full_path):
                img_path = full_path
                break
        
        if img_path is None:
            raise FileNotFoundError(f"Imagem não encontrada para ID {patient_id}")
        
        image = Image.open(img_path).convert('RGB')
        labels = torch.tensor([row[col] for col in self.disease_cols], dtype=torch.float32)
        
        if self.transform:
            image = self.transform(image)
        
        return image, labels, patient_id

class CropOnly(object):
        """Só cropping, SEM CLAHE"""
        
        def __call__(self, img):
            img = np.array(img)
            
            # Mesmo cropping que ApplyCLAHEandCrop_Adaptive mas SEM CLAHE
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            mask = gray > 10
            if np.any(mask):
                coords = np.argwhere(mask)
                y0, x0 = coords.min(axis=0)
                y1, x1 = coords.max(axis=0) + 1
                img = img[y0:y1, x0:x1]
            
            return Image.fromarray(img)
class ApplyCLAHEandCrop_Adaptive(object):
    """
    CLAHE adaptativo: só aplica em imagens de baixo contraste
    """
    
    def __init__(self):
        self.contrast_threshold = 50   # ← OTIMIZADO!
        self.clip_limit = 3.0          # Suave
        self.tile_grid_size = (8, 8)
    
    def __call__(self, img):
        # To numpy (opencv)
        img = np.array(img)
        
        # ==================== CROPPING ====================
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray > 10
        
        if np.any(mask):
            coords = np.argwhere(mask)
            y0, x0 = coords.min(axis=0)
            y1, x1 = coords.max(axis=0) + 1
            img = img[y0:y1, x0:x1]
        
        # ==================== MEDIR CONTRASTE ====================
        gray_cropped = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        contrast = gray_cropped.std()  # Standard deviation = contraste
        
        # ==================== CLAHE ADAPTATIVO ====================
        # SÓ aplicar se contraste baixo!
        if contrast < self.contrast_threshold:
            # Converter para LAB
            lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
            l, a, b = cv2.split(lab)
            
            # Aplicar CLAHE no canal L
            clahe = cv2.createCLAHE(
                clipLimit=self.clip_limit,
                tileGridSize=self.tile_grid_size
            )
            l = clahe.apply(l)
            
            # Merge e converter de volta
            img = cv2.merge([l, a, b])
            img = cv2.cvtColor(img, cv2.COLOR_LAB2RGB)
        
        # To PIL
        return Image.fromarray(img)

def get_train_transform():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

def get_train_transform_CLAHE():
    return transforms.Compose([
        ApplyCLAHEandCrop_Adaptive(),
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

def get_val_test_transform():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

def get_val_test_transform_CLAHE():
    return transforms.Compose([
        ApplyCLAHEandCrop_Adaptive(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
        
def get_v1_baseline_transform():
    """V1: Baseline puro (SEM crop, SEM aug, SEM CLAHE)"""
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])


def get_v2_crop_only_transform():
    """V2: Só cropping (SEM aug, SEM CLAHE)"""
    return transforms.Compose([
        CropOnly(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])


def get_v3_crop_aug_transform():
    """V3: Cropping + Augmentation (SEM CLAHE)"""
    return transforms.Compose([
        CropOnly(),
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=5),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])


def get_v4_full_pipeline_transform():
    """V4: Full Pipeline (Crop + Aug + CLAHE)"""
    return transforms.Compose([
        ApplyCLAHEandCrop_Adaptive(),
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
# --- Verificação de Segurança e Execução ---
if not os.path.exists(DATASET_ROOT):
    print(f"❌ ERRO: Dataset não encontrado em {DATASET_ROOT}")
else:
    # 1. Criar Datasets Reais
    train_dataset_basic = ODIRDataset('train', transform=get_train_transform())
    train_dataset_clahe = ODIRDataset('train', transform=get_train_transform_CLAHE())
    
    val_dataset = ODIRDataset('val', transform=get_val_test_transform_CLAHE())
    test_dataset = ODIRDataset('test', transform=get_val_test_transform_CLAHE())
    
    print("\n Resume: Train:", len(train_dataset_clahe), "| Val:", len(val_dataset), "| Test:", len(test_dataset))

def create_datasets_for_config(config_name):
    """
    Criar train/val/test datasets para uma configuração
    
    Args:
        config_name: 'v1', 'v2', 'v3', ou 'v4'
    
    Returns:
        dict com keys 'train', 'val', 'test'
    """
    
    train_transforms = {
        'v1': get_v1_baseline_transform(),
        'v2': get_v2_crop_only_transform(),
        'v3': get_v3_crop_aug_transform(),
        'v4': get_v4_full_pipeline_transform()
    }
    
    val_test_transforms = {
        'v1': get_val_test_transform(),
        'v2': get_val_test_transform(),
        'v3': get_val_test_transform(),
        'v4': get_val_test_transform_CLAHE()
    }
    
    print(f"\n📦 Criando datasets para config: {config_name.upper()}")
    
    datasets = {
        'train': ODIRDataset('train', transform=train_transforms[config_name]),
        'val': ODIRDataset('val', transform=val_test_transforms[config_name]),
        'test': ODIRDataset('test', transform=val_test_transforms[config_name])
    }
    
    print(f"✅ Datasets {config_name} criados!")
    
    return datasets

🚀 Dispositivo: cuda
  📂 train: 4474 pacients, 5732 images
  📂 train: 4474 pacients, 5732 images
  📂 val: 959 pacients, 1728 images
  📂 test: 959 pacients, 1748 images

 Resume: Train: 4474 | Val: 959 | Test: 959


In [3]:
datasets_v1 = create_datasets_for_config('v1')
datasets_v2 = create_datasets_for_config('v2')
datasets_v3 = create_datasets_for_config('v3')
datasets_v4 = create_datasets_for_config('v4')


📦 Criando datasets para config: V1
  📂 train: 4474 pacients, 5732 images
  📂 val: 959 pacients, 1728 images
  📂 test: 959 pacients, 1748 images
✅ Datasets v1 criados!

📦 Criando datasets para config: V2
  📂 train: 4474 pacients, 5732 images
  📂 val: 959 pacients, 1728 images
  📂 test: 959 pacients, 1748 images
✅ Datasets v2 criados!

📦 Criando datasets para config: V3
  📂 train: 4474 pacients, 5732 images
  📂 val: 959 pacients, 1728 images
  📂 test: 959 pacients, 1748 images
✅ Datasets v3 criados!

📦 Criando datasets para config: V4
  📂 train: 4474 pacients, 5732 images
  📂 val: 959 pacients, 1728 images
  📂 test: 959 pacients, 1748 images
✅ Datasets v4 criados!


In [4]:
train_v1 = datasets_v1['train']
val_v1 = datasets_v1['val']
test_v1 = datasets_v1['test']

train_v2 = datasets_v2['train']
val_v2 = datasets_v2['val']
test_v2 = datasets_v2['test']

train_v3 = datasets_v3['train']
val_v3 = datasets_v3['val']
test_v3 = datasets_v3['test']

train_v4 = datasets_v4['train']
val_v4 = datasets_v4['val']
test_v4 = datasets_v4['test']

In [5]:
class MobileNetV3Model(nn.Module):
    """
    MobileNetV3-Large
    - Moderno e eficiente
    - Rápido (2-3x mais que ResNet)
    - Funciona bem em medical imaging
    """
    
    def __init__(self, num_classes=8, dropout=0.2):
        super().__init__()
        
        # Carregar pretrained
        self.model = timm.create_model('mobilenetv3_large_100', pretrained=True)
        
        # Substituir classifier
        num_features = self.model.classifier.in_features
        
        self.model.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(num_features, num_classes)
        )
    
    def forward(self, x):
        return self.model(x)

In [6]:
print("\n📊 Model Information:")
model_test = MobileNetV3Model()
total_params = sum(p.numel() for p in model_test.parameters())
trainable_params = sum(p.numel() for p in model_test.parameters() if p.requires_grad)

print(f"  Total parameters:     {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Model size:           ~{total_params * 4 / 1024 / 1024:.1f} MB")

del model_test


📊 Model Information:


model.safetensors:   0%|          | 0.00/22.1M [00:00<?, ?B/s]

  Total parameters:     4,212,280
  Trainable parameters: 4,212,280
  Model size:           ~16.1 MB


In [7]:
print("\n📂 Using V4 preprocessing (CLAHE + Full Pipeline)...")

# Reusar datasets de V4
train_mobile = train_v4  # Mesmo preprocessing!
val_mobile = val_v4
test_mobile = test_v4

print(f"✅ Datasets ready:")
print(f"   Train: {len(train_mobile)} samples")
print(f"   Val:   {len(val_mobile)} samples")
print(f"   Test:  {len(test_mobile)} samples")


📂 Using V4 preprocessing (CLAHE + Full Pipeline)...
✅ Datasets ready:
   Train: 4474 samples
   Val:   959 samples
   Test:  959 samples


In [8]:
def train_model_complete(train_dataset, val_dataset, test_dataset, 
                        config_name, num_epochs=10, batch_size=32, lr=1e-4, use_class_weights=False, use_focal_loss=False):
    """
    Treinar MOBILENET-V3 com visualizações automáticas para o relatório
    """
    
    print(f"\n{'='*70}")
    print(f"TRAINING MOBILENET-V3 - {config_name.upper()}")
    print(f"{'='*70}\n")
    
    # Setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pin= True
    print(f"\n🚀 Device: {device}")
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True, num_workers=2, pin_memory=pin)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size,
                              shuffle=False, num_workers=2, pin_memory=pin)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size,
                              shuffle=False, num_workers=2, pin_memory=pin)

    # Model
    model = MobileNetV3Model()
    model = model.to(device)
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"📊 Model: {num_params:,} parameters")

    # ==================== LOSS COM CLASS WEIGHTS ====================
    if use_class_weights:
        print("\n🔧 Using CLIPPED CLASS WEIGHTS to balance classes...")
        class_weights = calculate_class_weights_clipped(  # ← USAR FUNÇÃO NOVA!
            train_dataset, 
            max_weight=3.0  # ← H será 2.38 (não 5.65!)
        ).to(device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)
        print("✅ Clipped class weights activated!")
    else:
        criterion = nn.BCEWithLogitsLoss()
        print("⚠️  Using standard BCE (no class weights)")

    criterion = criterion.to(device)
    
    # Optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr,           
        weight_decay=1e-4  
    )    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5
    )
    
    # Tracking
    history = {
        'train_loss': [], 'train_f1': [], 'train_acc': [],
        'val_loss': [], 'val_f1': [], 'val_acc': [],
        'learning_rates': [], 'epoch_times': []
    }
    
    best_val_f1 = 0.0
    start_time = time.time()
    
    # ==================== TRAINING LOOP ====================
    for epoch in range(num_epochs):
        epoch_start = time.time()
        
        # TRAIN
        model.train()
        train_loss = 0.0
        train_preds, train_labels = [], []
        
        for images, labels, _ in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]'):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad(set_to_none=True)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_preds.append(torch.sigmoid(outputs).detach().cpu())
            train_labels.append(labels.cpu())
        
        train_loss /= len(train_loader)
        train_preds = torch.cat(train_preds).numpy()
        train_labels = torch.cat(train_labels).numpy()
        train_preds_binary = (train_preds > 0.5).astype(int)
        
        train_f1 = f1_score(train_labels, train_preds_binary, average='macro', zero_division=0)
        train_acc = 1.0 - hamming_loss(train_labels, train_preds_binary)
        
        # VALIDATION
        model.eval()
        val_loss = 0.0
        val_preds, val_labels = [], []
        
        with torch.no_grad():
            for images, labels, _ in tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]'):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                val_preds.append(torch.sigmoid(outputs).cpu())
                val_labels.append(labels.cpu())
        
        val_loss /= len(val_loader)
        val_preds = torch.cat(val_preds).numpy()
        val_labels = torch.cat(val_labels).numpy()
        val_preds_binary = (val_preds > 0.5).astype(int)
        
        val_f1 = f1_score(val_labels, val_preds_binary, average='macro', zero_division=0)
        val_acc   = 1.0 - hamming_loss(val_labels, val_preds_binary)
        
        # Scheduler
        scheduler.step(val_f1)
        current_lr = optimizer.param_groups[0]['lr']
        
        # Save history
        epoch_time = time.time() - epoch_start
        history['train_loss'].append(train_loss)
        history['train_f1'].append(train_f1)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_f1'].append(val_f1)
        history['val_acc'].append(val_acc)
        history['learning_rates'].append(current_lr)
        history['epoch_times'].append(epoch_time)
        
        # Print progress
        print(f"\nEpoch {epoch+1}/{num_epochs}:")
        print(f"  Train - Loss: {train_loss:.4f}, F1: {train_f1:.4f}, Acc: {train_acc:.4f}")
        print(f"  Val   - Loss: {val_loss:.4f}, F1: {val_f1:.4f}, Acc: {val_acc:.4f}")
        print(f"  LR: {current_lr:.2e}, Time: {epoch_time:.1f}s")
        
        # Save best model
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_f1': val_f1,
                'history': history
            }, f'efficientnet_{config_name}_best.pth')
            print(f"  ✅ Best model saved! (F1: {val_f1:.4f})")
    
    total_train_time = time.time() - start_time
    
    # ==================== TEST EVALUATION ====================
    print(f"\n{'='*70}")
    print("EVALUATING ON TEST SET")
    print(f"{'='*70}\n")
    
    checkpoint = torch.load(f'efficientnet_{config_name}_best.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    test_preds, test_labels = [], []
    inference_times = []
    
    with torch.no_grad():
        for images, labels, _ in tqdm(test_loader, desc='Testing'):
            images, labels = images.to(device), labels.to(device)
            
            start = time.time()
            outputs = model(images)
            inference_times.append((time.time() - start) / len(images))
            
            test_preds.append(torch.sigmoid(outputs).cpu())
            test_labels.append(labels.cpu())
    
    test_preds = torch.cat(test_preds).numpy()
    test_labels = torch.cat(test_labels).numpy()
    test_preds_binary = (test_preds > 0.5).astype(int)
    
    # Metrics
    test_f1 = f1_score(test_labels, test_preds_binary, average='macro', zero_division=0)
    test_acc  = 1.0 - hamming_loss(test_labels, test_preds_binary)
    kappas = [
        cohen_kappa_score(test_labels[:, i], test_preds_binary[:, i])
        for i in range(test_labels.shape[1])
    ]
    test_kappa = float(np.mean(kappas))
    
    # Per-class metrics
    class_names = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
    per_class_f1 = f1_score(test_labels, test_preds_binary, average=None, zero_division=0)
    
    print(f"\n📊 FINAL TEST RESULTS - {config_name.upper()}:")
    print(f"  Accuracy:  {test_acc:.4f}")
    print(f"  F1-Score:  {test_f1:.4f}")
    print(f"  Kappa:     {test_kappa:.4f}")
    print(f"  Avg Epoch Time: {np.mean(history['epoch_times']):.1f}s")
    print(f"  Avg Inference:  {np.mean(inference_times)*1000:.2f}ms/image")
    print(f"\n  Per-class F1:")
    for i, name in enumerate(class_names):
        print(f"    {name}: {per_class_f1[i]:.4f}")
    
    # ==================== SAVE RESULTS ====================
    results = {
        'config': config_name,
        'num_params': num_params,
        'num_epochs': num_epochs,
        'best_val_f1': float(best_val_f1),
        'test_accuracy': float(test_acc),
        'test_f1': float(test_f1),
        'test_kappa': float(test_kappa),
        'per_class_f1': {name: float(f1) for name, f1 in zip(class_names, per_class_f1)},
        'total_train_time': float(total_train_time),
        'avg_epoch_time': float(np.mean(history['epoch_times'])),
        'avg_inference_time_ms': float(np.mean(inference_times) * 1000)
    }
    
    with open(f'efficientnet_{config_name}_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    # Save history
    np.save(f'efficientnet_{config_name}_history.npy', history)
    
    # ==================== GENERATE VISUALIZATIONS ====================
    print(f"\n📊 Generating visualizations for report...")
    
    # 1. TRAINING CURVES
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'EfficientNet-B0 {config_name.upper()} - Training Progress', 
                 fontsize=16, fontweight='bold')
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss
    axes[0, 0].plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
    axes[0, 0].plot(epochs, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Training & Validation Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # F1-Score
    axes[0, 1].plot(epochs, history['train_f1'], 'b-', label='Train F1', linewidth=2)
    axes[0, 1].plot(epochs, history['val_f1'], 'r-', label='Val F1', linewidth=2)
    axes[0, 1].axhline(y=best_val_f1, color='g', linestyle='--', 
                       label=f'Best Val F1: {best_val_f1:.4f}')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('F1-Score')
    axes[0, 1].set_title('F1-Score Progress')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    # Accuracy
    axes[1, 0].plot(epochs, history['train_acc'], 'b-', label='Train Acc', linewidth=2)
    axes[1, 0].plot(epochs, history['val_acc'], 'r-', label='Val Acc', linewidth=2)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].set_title('Accuracy Progress')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    # Learning Rate
    axes[1, 1].plot(epochs, history['learning_rates'], 'purple', linewidth=2)
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Learning Rate')
    axes[1, 1].set_title('Learning Rate Schedule')
    axes[1, 1].set_yscale('log')
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'efficientnet_{config_name}_training_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. CONFUSION MATRIX (multi-label)
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    fig.suptitle(f'{config_name.upper()} - Per-Class Confusion Matrices', 
                 fontsize=16, fontweight='bold')
    
    for i, class_name in enumerate(class_names):
        ax = axes[i // 4, i % 4]
        cm = confusion_matrix(test_labels[:, i], test_preds_binary[:, i])
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, 
                   xticklabels=['Neg', 'Pos'], yticklabels=['Neg', 'Pos'])
        ax.set_title(f'{class_name} (F1: {per_class_f1[i]:.3f})')
        ax.set_xlabel('Predicted')
        ax.set_ylabel('True')
    
    plt.tight_layout()
    plt.savefig(f'efficientnet_{config_name}_confusion_matrices.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 3. PER-CLASS F1 BAR CHART
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(class_names, per_class_f1, color='steelblue', edgecolor='black')
    ax.axhline(y=test_f1, color='red', linestyle='--', linewidth=2, 
               label=f'Average F1: {test_f1:.4f}')
    ax.set_xlabel('Disease Class', fontsize=12)
    ax.set_ylabel('F1-Score', fontsize=12)
    ax.set_title(f'{config_name.upper()} - Per-Class Performance', 
                 fontsize=14, fontweight='bold')
    ax.set_ylim([0, 1.0])
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, f1 in zip(bars, per_class_f1):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{f1:.3f}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(f'efficientnet_{config_name}_per_class_f1.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✅ Visualizations saved:")
    print(f"   - efficientnet_{config_name}_training_curves.png")
    print(f"   - efficientnet_{config_name}_confusion_matrices.png")
    print(f"   - efficientnet_{config_name}_per_class_f1.png")
    print(f"   - efficientnet_{config_name}_results.json")
    print(f"   - efficientnet_{config_name}_history.npy")
    
    return results, model, history

print("✅ Training function with visualizations ready!")

✅ Training function with visualizations ready!


In [9]:
print("\n Starting training...")
print("="*80 + "\n")

# TREINAR (reusar função existente)
results_mobile, model_mobile, history_mobile = train_model_complete(
    train_mobile, 
    val_mobile, 
    test_mobile,
    config_name='mobilenetv3_large',
    num_epochs=40,
    batch_size=32,
    lr=1e-4,
    use_class_weights=False,
    use_focal_loss=False
)


 Starting training...


TRAINING MOBILENET-V3 - MOBILENETV3_LARGE


🚀 Device: cuda
📊 Model: 4,212,280 parameters
⚠️  Using standard BCE (no class weights)


Epoch 1/40 [Val]: 100%|██████████| 30/30 [00:13<00:00,  2.15it/s]



Epoch 1/40:
  Train - Loss: 0.3604, F1: 0.0869, Acc: 0.8499
  Val   - Loss: 0.3012, F1: 0.2299, Acc: 0.8693
  LR: 1.00e-04, Time: 78.0s
  ✅ Best model saved! (F1: 0.2299)


Epoch 2/40 [Val]: 100%|██████████| 30/30 [00:10<00:00,  2.83it/s]



Epoch 2/40:
  Train - Loss: 0.2856, F1: 0.2934, Acc: 0.8723
  Val   - Loss: 0.2711, F1: 0.3844, Acc: 0.8797
  LR: 1.00e-04, Time: 63.1s
  ✅ Best model saved! (F1: 0.3844)


...


Epoch 39/40:
  Train - Loss: 0.0070, F1: 0.9927, Acc: 0.9977
  Val   - Loss: 0.2996, F1: 0.8441, Acc: 0.9492
  LR: 5.00e-05, Time: 68.0s


Epoch 40/40 [Val]: 100%|██████████| 30/30 [00:10<00:00,  2.77it/s]



Epoch 40/40:
  Train - Loss: 0.0083, F1: 0.9916, Acc: 0.9972
  Val   - Loss: 0.2988, F1: 0.8505, Acc: 0.9496
  LR: 5.00e-05, Time: 67.0s

EVALUATING ON TEST SET



Testing: 100%|██████████| 30/30 [00:15<00:00,  1.98it/s]



📊 FINAL TEST RESULTS - MOBILENETV3_LARGE:
  Accuracy:  0.9572
  F1-Score:  0.8574
  Kappa:     0.8287
  Avg Epoch Time: 65.5s
  Avg Inference:  0.38ms/image

  Per-class F1:
    N: 0.8624
    D: 0.8784
    G: 0.8480
    C: 0.9167
    A: 0.8571
    H: 0.8000
    M: 0.9286
    O: 0.7679

📊 Generating visualizations for report...
✅ Visualizations saved:
   - efficientnet_mobilenetv3_large_training_curves.png
   - efficientnet_mobilenetv3_large_confusion_matrices.png
   - efficientnet_mobilenetv3_large_per_class_f1.png
   - efficientnet_mobilenetv3_large_results.json
   - efficientnet_mobilenetv3_large_history.npy


In [10]:
print("\n" + "="*80)
print("📊 MOBILENETV3 vs V4 COMPARISON")
print("="*80)

mobile_f1 = results_mobile['test_f1']
v4_f1 = 0.8659  # V4 default

print(f"\n🏆 TEST F1-SCORES:")
print(f"  V4 (EfficientNet-B0):      {v4_f1:.4f}")
print(f"  MobileNetV3-Large:         {mobile_f1:.4f}")
print(f"  Difference:                {mobile_f1 - v4_f1:+.4f} ({(mobile_f1/v4_f1 - 1)*100:+.1f}%)")


📊 MOBILENETV3 vs V4 COMPARISON

🏆 TEST F1-SCORES:
  V4 (EfficientNet-B0):      0.8659
  MobileNetV3-Large:         0.8574
  Difference:                -0.0085 (-1.0%)


In [11]:
if mobile_f1 > 0.85:
    print(f"\n{'='*80}")
    print("✅ MOBILENETV3 PERFORMANCE GOOD!")
    print(f"{'='*80}")
    print(f"MobileNetV3 achieved F1 > 0.85: {mobile_f1:.4f}")
    print("Ensemble with V4 is RECOMMENDED!")
    print("\nExpected ensemble F1: 0.87-0.88")
    print("Proceed to ensemble cell below!")
    
elif mobile_f1 > 0.80:
    print(f"\n{'='*80}")
    print("🤔 MOBILENETV3 PERFORMANCE OK")
    print(f"{'='*80}")
    print(f"MobileNetV3 achieved F1 = {mobile_f1:.4f}")
    print("Ensemble MIGHT help, but uncertain.")
    print("Test ensemble to see if improves over V4.")
    
else:
    print(f"\n{'='*80}")
    print("⚠️  MOBILENETV3 PERFORMANCE WEAK")
    print(f"{'='*80}")
    print(f"MobileNetV3 achieved F1 = {mobile_f1:.4f}")
    print("Ensemble NOT recommended (would hurt performance).")
    print("Use V4 default (0.8659) as final result.")

print("="*80)


✅ MOBILENETV3 PERFORMANCE GOOD!
MobileNetV3 achieved F1 > 0.85: 0.8574
Ensemble with V4 is RECOMMENDED!

Expected ensemble F1: 0.87-0.88
Proceed to ensemble cell below!


In [12]:
import json

comparison_results = {
    'v4_efficientnet': {
        'test_f1': v4_f1,
        'config': 'EfficientNet-B0 + CLAHE + Full Pipeline'
    },
    'mobilenetv3': {
        'test_f1': mobile_f1,
        'config': 'MobileNetV3-Large + CLAHE + Full Pipeline',
        'per_class_f1': results_mobile['per_class_f1']
    },
    'comparison': {
        'difference': float(mobile_f1 - v4_f1),
        'percent_change': float((mobile_f1/v4_f1 - 1)*100),
        'best_model': 'mobilenetv3' if mobile_f1 > v4_f1 else 'v4'
    }
}

with open('/kaggle/working/model_comparison_v4_vs_mobile.json', 'w') as f:
    json.dump(comparison_results, f, indent=2)

print("\n✅ Comparison saved: model_comparison_v4_vs_mobile.json")


✅ Comparison saved: model_comparison_v4_vs_mobile.json
